In [11]:
import torch
import torch.nn as nn
import os
import math
from typing import List, Tuple
# !pip install sacrebleu tqdm -q
import sacrebleu
from tqdm import tqdm

# Google Drive'a bağlan
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# --- Model ve Tokenizer Sınıf Tanımları ---
class WordTokenizer:
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0
    def encode(self, sentence: str, add_sos_eos: bool = False) -> List[int]:
        tokens = [self.word2idx.get(word, self.word2idx['<unk>']) for word in sentence.lower().split()]
        if add_sos_eos: tokens = [self.word2idx['<sos>']] + tokens + [self.word2idx['<eos>']]
        return tokens
    def decode(self, indices: List[int]) -> str:
        return ' '.join([self.idx2word.get(idx, '<unk>') for idx in indices if idx not in [self.word2idx['<pad>'], self.word2idx['<sos>'], self.word2idx['<eos>']]])

class Embedding(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int, padding_idx: int = None):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim, padding_idx=padding_idx)
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.embedding(x)

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=100):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)[:,:d_model//2] if d_model % 2 != 0 else torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1)])

class Transformer(nn.Module):
    def __init__(self, src_vocab_size: int, trg_vocab_size: int, src_pad_idx: int, trg_pad_idx: int,
                 d_model: int, nhead: int, num_encoder_layers: int, num_decoder_layers: int,
                 dim_feedforward: int, dropout: float, activation_fn_str: str, max_seq_len: int):
        super().__init__()
        self.d_model = d_model
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.src_embedding = Embedding(src_vocab_size, d_model, padding_idx=src_pad_idx)
        self.trg_embedding = Embedding(trg_vocab_size, d_model, padding_idx=trg_pad_idx)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=max_seq_len)
        self.transformer = nn.Module()
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation_fn_str, batch_first=True, norm_first=False)
        self.transformer.encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers, nn.LayerNorm(d_model))
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, activation_fn_str, batch_first=True, norm_first=False)
        self.transformer.decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers, nn.LayerNorm(d_model))
        self.fc_out = nn.Linear(d_model, trg_vocab_size)
    def generate_square_subsequent_mask(self, sz: int, device: torch.device) -> torch.Tensor:
        return torch.triu(torch.full((sz, sz), float('-inf'), device=device, dtype=torch.float32), diagonal=1)
    def create_padding_mask(self, seq: torch.Tensor, pad_idx: int) -> torch.Tensor:
        return (seq == pad_idx)
    def forward(self, src: torch.Tensor, trg_input: torch.Tensor) -> torch.Tensor:
        src_emb = self.pos_encoder(self.src_embedding(src) * math.sqrt(self.d_model))
        trg_emb = self.pos_encoder(self.trg_embedding(trg_input) * math.sqrt(self.d_model))
        src_key_padding_mask_bool = self.create_padding_mask(src, self.src_pad_idx)
        trg_key_padding_mask_bool = self.create_padding_mask(trg_input, self.trg_pad_idx)
        tgt_attn_mask_float = self.generate_square_subsequent_mask(trg_input.size(1), device=trg_input.device)
        src_key_padding_mask_float = torch.zeros_like(src_key_padding_mask_bool, dtype=src_emb.dtype).masked_fill_(src_key_padding_mask_bool, float('-inf'))
        trg_key_padding_mask_float = torch.zeros_like(trg_key_padding_mask_bool, dtype=trg_emb.dtype).masked_fill_(trg_key_padding_mask_bool, float('-inf'))
        memory = self.transformer.encoder(src_emb, mask=None, src_key_padding_mask=src_key_padding_mask_float)
        output = self.transformer.decoder(trg_emb, memory, tgt_mask=tgt_attn_mask_float, memory_mask=None,
                                          tgt_key_padding_mask=trg_key_padding_mask_float,
                                          memory_key_padding_mask=src_key_padding_mask_float)
        return self.fc_out(output)

@torch.no_grad()
def translate_single_sentence(
    inference_model: Transformer, sentence_str: str, src_tokenizer_inf: WordTokenizer,
    trg_tokenizer_inf: WordTokenizer, device_inf: torch.device, max_len_inf: int):
    inference_model.eval()
    src_tokens = src_tokenizer_inf.encode(sentence_str.strip().lower(), add_sos_eos=True)
    src_tensor = torch.LongTensor(src_tokens).unsqueeze(0).to(device_inf)
    trg_indices = [trg_tokenizer_inf.word2idx['<sos>']]
    for _ in range(max_len_inf):
        trg_tensor = torch.LongTensor(trg_indices).unsqueeze(0).to(device_inf)
        output_logits = inference_model(src_tensor, trg_tensor)
        pred_token = output_logits.argmax(2)[:, -1].item()
        trg_indices.append(pred_token)
        if pred_token == trg_tokenizer_inf.word2idx['<eos>']: break
    return trg_tokenizer_inf.decode(trg_indices)

def load_sentence_pairs_for_eval(filepath: str) -> tuple[list[str], list[str]]:
    sources, targets = [], []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                sources.append(parts[0])
                targets.append(parts[1])
    return sources, targets

def run_bleu_evaluation():
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    DRIVE_BASE_PATH = '/content/drive/My Drive/transformer/'
    CHECKPOINT_DIR = os.path.join(DRIVE_BASE_PATH, 'model_checkpoints')

    CONFIG_CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, 'last_checkpoint_512.pth')
    BEST_MODEL_WEIGHTS_PATH = os.path.join(CHECKPOINT_DIR, 'best_model_512.pth')
    VALIDATION_FILE_PATH = os.path.join(DRIVE_BASE_PATH, 'en2tr_valid.txt')

    print("--- BLEU Skoru Değerlendirmesi Başlatılıyor ---")
    if not all(os.path.exists(p) for p in [CONFIG_CHECKPOINT_PATH, BEST_MODEL_WEIGHTS_PATH, VALIDATION_FILE_PATH]):
        print("HATA: Gerekli dosyalardan biri bulunamadı. Yolları kontrol edin.")
        return

    # 1. Yapılandırmayı ve tokenizer'ları yükle
    print("Model yapılandırması ve tokenizer'lar yükleniyor...")
    checkpoint = torch.load(CONFIG_CHECKPOINT_PATH, map_location='cpu')

    src_tokenizer = WordTokenizer(); src_tokenizer.word2idx = checkpoint['src_word2idx']; src_tokenizer.idx2word = {v:k for k,v in checkpoint['src_word2idx'].items()}
    trg_tokenizer = WordTokenizer(); trg_tokenizer.word2idx = checkpoint['trg_word2idx']; trg_tokenizer.idx2word = {v:k for k,v in checkpoint['trg_word2idx'].items()}

    # Transformer'ın ihtiyaç duyduğu anahtarları bir listede tanımlayalım
    model_architecture_keys = [
        'd_model', 'nhead', 'num_encoder_layers', 'num_decoder_layers',
        'dim_feedforward', 'dropout', 'activation_fn_str', 'max_seq_len'
    ]
    # Sadece bu anahtarları checkpoint'ten alarak config sözlüğünü oluşturalım
    config = {key: checkpoint[key] for key in model_architecture_keys if key in checkpoint}

    pad_idx = checkpoint.get('pad_idx', 0)
    config['src_pad_idx'] = pad_idx
    config['trg_pad_idx'] = pad_idx

    # 2. Modeli oluştur ve en iyi ağırlıkları yükle
    print("En iyi model ağırlıkları yükleniyor...")
    model = Transformer(
        src_vocab_size=len(src_tokenizer.word2idx),
        trg_vocab_size=len(trg_tokenizer.word2idx),
        **config
    ).to(DEVICE)

    model.load_state_dict(torch.load(BEST_MODEL_WEIGHTS_PATH, map_location=DEVICE, weights_only=True))
    model.eval()

    print("Model başarıyla yüklendi.")
    sources, references = load_sentence_pairs_for_eval(VALIDATION_FILE_PATH)
    print(f"{len(sources)} cümle için çeviri üretiliyor... Bu işlem uzun sürebilir.")
    hypotheses = [translate_single_sentence(model, src, src_tokenizer, trg_tokenizer, DEVICE, config.get('max_seq_len', 50)) for src in tqdm(sources)]
    bleu = sacrebleu.corpus_bleu(hypotheses, [references])

    print("\n" + "="*50); print(" " * 15 + "BLEU SKORU SONUCU"); print("="*50)
    print(bleu)
    print("="*50)

run_bleu_evaluation()


Mounted at /content/drive
--- BLEU Skoru Değerlendirmesi Başlatılıyor ---
Model yapılandırması ve tokenizer'lar yükleniyor...


<ipython-input-11-46c402a060b0>:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CONFIG_CHECKPOINT_PATH, map_location='cpu')


En iyi model ağırlıkları yükleniyor...
Model başarıyla yüklendi.
47304 cümle için çeviri üretiliyor... Bu işlem uzun sürebilir.


  0%|          | 0/47304 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:502: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  output = torch._nested_tensor_from_mask(
100%|██████████| 47304/47304 [54:00<00:00, 14.60it/s]



               BLEU SKORU SONUCU
BLEU = 46.13 70.4/52.1/39.9/30.9 (BP = 1.000 ratio = 1.045 hyp_len = 236179 ref_len = 226086)
